In [99]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [100]:
# Function to calculate average price by location
def calculate_average_price_by_location(data):
    average_prices = data.groupby('Location')['Price'].mean().reset_index()
    return average_prices

In [101]:
# Setup Selenium WebDriver 
driver = webdriver.Chrome() 

In [102]:
# Navigate to ikman.lk
driver.get("https://ikman.lk/")

In [ ]:
try:
    # Wait for the search box and search button to be located
    search_box = WebDriverWait(driver, 60).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'input[name="q"]'))
    )
    search_button = WebDriverWait(driver, 60).until(
        EC.visibility_of_element_located((By.CLASS_NAME, "search-box__button"))
    )

    # Search for motorcycle model "CT-100"
    search_box.send_keys("CT-100")
    search_button.click()

    # Initialize lists to store data
    prices = []
    locations = []
    years = []
    mileages = []

    # Extract information from the first 50 results
    for result in driver.find_elements(By.CLASS_NAME, "result-item")[:50]:
        price = result.find_element(By.CLASS_NAME, "price").text
        location = result.find_element(By.CLASS_NAME, "location").text
        year = result.find_element(By.CLASS_NAME, "date").text
        mileage_element = result.find_element(By.CLASS_NAME, "mileage")
        mileage = mileage_element.text if mileage_element else ""
        
        # Append extracted data to lists
        prices.append(price)
        locations.append(location)
        years.append(year)
        mileages.append(mileage)

    # Create DataFrame from extracted data
    data = pd.DataFrame({
        'Location': locations,
        'Price': prices,
        'Year of Manufacture': years,
        'Mileage': mileages
    })

    # Save data as CSV
    data.to_csv('ikman_ct100_listings.csv', index=False)

    # Calculate average price by location
    average_prices = calculate_average_price_by_location(data)

    # Display average prices
    print("Average Vehicle Price by Location:\n", average_prices)

except TimeoutException as e:
    print("TimeoutException occurred: Unable to locate elements within the specified timeout period.")

finally:
    # Close the WebDriver
    driver.quit()